In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
data = range(1,30)
# print first element of iterator
print(data[0])
len(data)
xrangeRDD = sc.parallelize(data,4)

# this will let us know that we created an RDD
xrangeRDD.first

1


<bound method RDD.first of PythonRDD[1] at RDD at PythonRDD.scala:53>

In [7]:
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<10)
print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


10

In [6]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# first count will trigger evaluation of count *and* cache
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# second count operates on cached data only
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

dt1:  0.9338979721069336
dt2:  0.23352432250976562


In [8]:
#In order to work with the extremely powerful SQL engine in Apache Spark, you will need a Spark Session. We have created that in the first Exercise, let us verify that spark session is still active.
spark

In [9]:
# Download the data first into a local `people.json` file
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    73    0     0    165      0 --:--:-- --:--:-- --:--:--   165


In [10]:
# Read the dataset into a spark dataframe using the `read.json()` function
df = spark.read.json("people.json").cache()

In [11]:
# Print the dataframe as well as the data schema
df.show()
df.printSchema()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
# Register the DataFrame as a SQL temporary view
df.createTempView("people")

In [13]:
# Select and show basic data columns
t1=time.time()
df.select("name").show()
t2=time.time()-t1
print(t2)
t3=time.time()
df.select(df["name"]).show()
t4=time.time()-t3
print(t4)
t5=time.time()
spark.sql("SELECT name FROM people").show()
t6=time.time()-t5
print(t6)

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

0.4233231544494629
+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

0.20926642417907715
+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

0.4521629810333252


In [14]:
df.filter(df["age"] > 21).show()
spark.sql("SELECT age, name FROM people WHERE age > 21").show()


+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [15]:
df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    0|
|  30|    1|
+----+-----+

